In [1]:
import datetime
import pandas as pd
import numpy as np
from btyd import BetaGeoModel
import pickle

RFM_PATH = 'data\\rfm.parquet.gzip'
MODEL_PATH = "model\\rfm.model.pkl"

WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
rfm = pd.read_parquet(RFM_PATH)

In [3]:
model = pickle.load(open(MODEL_PATH, "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'model\\rfm_model.pkl'

In [ ]:
def conditional_probability_alive_matrix(
    model, data, max_frequency=None, max_recency=None
):

    max_frequency = max_frequency or int(data["frequency"].max())
    max_recency = max_recency or int(data["T"].max())

    return np.fromfunction(
        model.conditional_probability_alive,
        (max_frequency + 1, max_recency + 1),
        T=max_recency,
    ).T

In [ ]:
def plot_probability_alive_matrix(
    model,
    conditional_probability_alive,
    max_frequency=None,
    max_recency=None,
    title="Probability Customer is Alive,\nby Frequency and Recency of a Customer",
    xlabel="Customer's Historical Frequency",
    ylabel="Customer's Recency",
    **kwargs
):

    from matplotlib import pyplot as plt

    z = conditional_probability_alive

    interpolation = kwargs.pop("interpolation", "none")

    ax = plt.subplot(111)
    pcm = ax.imshow(z, interpolation=interpolation, **kwargs)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    forceAspect(ax)

    plt.colorbar(pcm, ax=ax)

    return ax


def forceAspect(ax, aspect=1):
    im = ax.get_images()
    extent = im[0].get_extent()
    ax.set_aspect(abs((extent[1] - extent[0]) / (extent[3] - extent[2])) / aspect)

In [ ]:
conditional_probability_alive = conditional_probability_alive_matrix(model, rfm)
plot_probability_alive_matrix(model, conditional_probability_alive)